In [1]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import pyarrow
from pandas import Timestamp
import itertools
from datetime import datetime
import scipy.stats as stats
import matplotlib.pyplot as plt 
from collections import Counter
from itertools import groupby
pd.set_option('display.max_rows', 500)
import matplotlib
import statistics 
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import scipy.io as sio
from scipy import fft, arange, signal
from scipy.stats import wasserstein_distance
from sklearn.ensemble import IsolationForest
plt.style.use('seaborn-poster')
%matplotlib inline

/var/folders/f8/cqx2q1h55yz_6tdkrbrd9y4r0000gn/T/ipykernel_2923/1954643802.py:23: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-poster')


In [2]:
# my credential file
#key_path = "/Users/yuxuanyang/Downloads/encoded-adviser-131008-f2e09e85c3b3.json"
key_path = "/Users/yuxuanyang/Downloads/encoded-adviser-131008-7129911936a3.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [3]:
#从bigquery中调取数据 用户的使用的access log和app的情报
class get_info_from_bigquery(object):
    def access_log(self, table_name):
        query = '\n    SELECT * FROM `encoded-adviser-131008.yang_20220414.{}` \n' .format(table_name)
        query_job = client.query(query)  # Make an API request.
        table = query_job.to_dataframe().sort_values(by=['user_id'])
        setattr(self, '{}'.format(table_name),table)
        #return table
    def original_table(self, table_name):
        query = '\n    SELECT * FROM `encoded-adviser-131008.yang_20220414.{}` \n' .format(table_name)
        query_job = client.query(query)  # Make an API request.
        table = query_job.to_dataframe()
        setattr(self, '{}'.format(table_name),table)

In [4]:
bigquery_class = get_info_from_bigquery()
#bigquery_class.access_log('200user_genre_datetime_accesscount')

In [5]:
bigquery_class.access_log('2000user_app_datetime_accesscount')

In [6]:
bigquery_class.original_table('2000user_app_genre')

In [7]:
#得到用户的accesslog和app的信息
user2000_app_datetime_accesscount = getattr(bigquery_class, '2000user_app_datetime_accesscount')
user2000_app_genre = getattr(bigquery_class, '2000user_app_genre')

In [8]:
%store user2000_app_datetime_accesscount
%store user2000_app_genre

Stored 'user2000_app_datetime_accesscount' (DataFrame)
Stored 'user2000_app_genre' (DataFrame)


In [12]:
app_list = user2000_app_datetime_accesscount[user2000_app_datetime_accesscount['user_id'].isin(user_id_list1)]['app_id'].unique().tolist()
X=user2000_app_genre[user2000_app_genre['app_id'].isin(app_list)]['genre'].unique()
len(X)

NameError: name 'user_id_list1' is not defined

In [14]:
#后一个与前一个元素的差值
def diff_between_after(lst):
    diff_list=[]
    for i in range(len(lst)-1):
        diff=lst[i+1]-lst[i]
        diff_list.append(diff)   
    return diff_list

In [15]:
def xstr(Xlist):
    new_Xlist = []
    for s in Xlist:
        if s is None:
            new_Xlist.append('NONE')
        else:
            new_Xlist.append(s)
    return new_Xlist

In [16]:
gen_table = user2000_app_genre 
gen_list = xstr(gen_table['genre'].unique())
all_ginterval=[]
for g in gen_list:
    gen_app_list = gen_table[gen_table['genre']==g]['app_id']
    alluser_ginterval = []
    for u in user_id_list1[:10]:
        user_app_list = getattr(user2000_new_interval, '{}_app_list'.format(u))
        common_app_list =[i for i in gen_app_list if i in user_app_list]
        if len(common_app_list)==[]:
            ginterval=[]
        else:
            ginterval=[]
            for a in common_app_list:
                ginterval += getattr(user2000_new_interval, '{}_{}_interval_list'.format(u,a))
        alluser_ginterval += ginterval
    all_ginterval.append(alluser_ginterval)
len(all_ginterval)

NameError: name 'user200_app_genre' is not defined

In [ ]:
user200_app_genre 

In [19]:
access_log = getattr(bigquery_class, '2000user_app_datetime_accesscount')
access_log[access_log['user_id']== '10a7a46ce3d1e7c61549a9979b5b78f4662f4438'].sort_values(by=['datetime']) #.unique()

,user_id,app_id,datetime,access_count
3699838,10a7a46ce3d1e7c61549a9979b5b78f4662f4438,com.com2us.smon.normal.freefull.google.kr.andr...,2020-09-30 03:59:13,1
1509739,10a7a46ce3d1e7c61549a9979b5b78f4662f4438,com.gmo_click.clickcfd,2020-09-30 04:03:53,1
1559632,10a7a46ce3d1e7c61549a9979b5b78f4662f4438,com.gmo_click.clickcfd,2020-09-30 04:04:00,1
1960749,10a7a46ce3d1e7c61549a9979b5b78f4662f4438,com.gmo_click.clickcfd,2020-09-30 04:04:04,1
3288451,10a7a46ce3d1e7c61549a9979b5b78f4662f4438,com.gmo_click.clickcfd,2020-09-30 04:04:05,1
3370915,10a7a46ce3d1e7c61549a9979b5b78f4662f4438,com.gmo_click.clickcfd,2020-09-30 04:04:28,1
4342949,10a7a46ce3d1e7c61549a9979b5b78f4662f4438,com.com2us.smon.normal.freefull.google.kr.andr...,2020-09-30 04:07:54,1
4009598,10a7a46ce3d1e7c61549a9979b5b78f4662f4438,com.com2us.smon.normal.freefull.google.kr.andr...,2020-09-30 04:25:05,1
4040420,10a7a46ce3d1e7c61549a9979b5b78f4662f4438,jp.co.incrementp.milemobile,2020-09-30 07:00:27,1
3284443,10a7a46ce3d1e7c61549a9979b5b78f4662f4438,jp.co.incrementp.milemobile,2020-09-30 07:01:02,1


In [ ]:
# 将一段时间内手机使用log转换成interval时长序列 (连续启动之间的时间间隔)
# 主要目的是查看每个app的interval时间长度分布
class new_access_interval_from_log(object):
    def access_log_to_new_interval(self, app_or_genre, user_id):  
        access_log =  getattr(bigquery_class, '2000user_{}_datetime_accesscount'.format(app_or_genre))
        access_timestamp_table_ = access_log[access_log['user_id']== user_id].sort_values(by=['datetime'])
        app_list = access_timestamp_table_['{}_id'.format(app_or_genre)].iloc[:-1].unique()
        #某个用户的a使用过的applist
        setattr(self,'{}_{}_list'.format(user_id,app_or_genre) ,app_list)
        app_order_list = access_timestamp_table_.sort_values(by='datetime').loc[:,'{}_id'.format(app_or_genre)].tolist()
        timestamp_list = access_timestamp_table_.sort_values(by='datetime').loc[:,'datetime'].tolist()
        count_list = access_timestamp_table_.sort_values(by='datetime').loc[:,'access_count'].tolist()
        setattr(self,'{}_{}_order_list'.format(user_id, app_or_genre) ,app_order_list)  
        setattr(self,'{}_timestamp_list'.format(user_id) ,timestamp_list)
        setattr(self,'{}_count_list'.format(user_id) ,count_list)
        duration_list = [(timestamp_list[i] - timestamp_list[0]).total_seconds() for i in range(len(timestamp_list))]
        setattr(self, '{}_duration_list'.format(user_id), duration_list)
        total_interval_list = diff_between_after(duration_list)
        setattr(self,'{}_total_interval_list'.format(user_id), total_interval_list)
        app_interval_list = []
        l = len(total_interval_list)
        for a in np.unique(np.array(app_order_list[:-1])).tolist():
        #找到某个使用appA的时点，转化成序号
            idx_lst = [i for i,t in enumerate(app_order_list[:l]) if t == a]
        #找到app A的使用时点序号
            app_duration = np.array(duration_list)[idx_lst]
            app_count = np.array(count_list)[idx_lst]
        #找到属于app A的使用时间间隔
            app_interval = np.array(total_interval_list)[idx_lst]
        #ユーザがアプリAを起動する時の時点
            setattr(self, '{}_{}_access_duration_list'.format(user_id,a), app_duration.tolist())
        #ユーザがアプリAを起動する回数
            setattr(self, '{}_{}_access_count_list'.format(user_id,a), app_count.tolist())
        #ユーザがアプリAの使用时间间隔
            setattr(self, '{}_{}_interval_list'.format(user_id,a), app_interval.tolist())
            app_interval_list.append(app_interval)
        setattr(self, '{}_app_interval_list'.format(user_id), app_interval_list)
        

In [18]:
%%time 
user200_new_interval = new_access_interval_from_log()
#%store -r sample_user_10per_202108
user_id_list1 = getattr(bigquery_class, '2000user_{}_datetime_accesscount'.format('app'))['user_id'].unique()
for u in user_id_list1:
     user200_new_interval.access_log_to_new_interval('app', u)
os.system('say "your program was finished"')
# del user200_genre_datetime_accesscount

KeyboardInterrupt: 

In [37]:
getattr(bigquery_class, '2000user_{}_datetime_accesscount'.format('app'))

,user_id,app_id,datetime,access_count
4501528,000e782b64c303f689d58dd3372be5fd2209aa9f,com.sec.android.app.launcher,2020-09-11 12:44:02,1
14073127,000e782b64c303f689d58dd3372be5fd2209aa9f,com.sec.android.app.launcher,2020-09-26 07:02:55,1
3212011,000e782b64c303f689d58dd3372be5fd2209aa9f,com.android.chrome,2020-09-11 11:01:17,1
16383060,000e782b64c303f689d58dd3372be5fd2209aa9f,com.sec.android.app.launcher,2020-09-17 05:15:40,1
16383061,000e782b64c303f689d58dd3372be5fd2209aa9f,com.sec.android.app.launcher,2020-09-17 05:17:22,1
...,...,...,...,...
2627561,ffa65bbc3da06d5f182ef6b69f8fd26586d12756,org.mozilla.fenix,2020-09-12 00:15:31,1
9725433,ffa65bbc3da06d5f182ef6b69f8fd26586d12756,com.sec.android.app.voicenote,2020-09-02 16:16:31,1
2627397,ffa65bbc3da06d5f182ef6b69f8fd26586d12756,org.mozilla.fenix,2020-09-12 00:09:16,1
12085644,ffa65bbc3da06d5f182ef6b69f8fd26586d12756,com.samsung.android.lool,2020-09-04 12:18:25,1


In [40]:
getattr(user200_new_interval,'{}_app_interval_list'.format(user_id_list1[2]))

[array([  0.,  22.,  31., ..., 118., 165., 124.]),
 array([1.7540e+03, 3.7000e+01, 9.0000e+00, 5.3000e+01, 4.0000e+00,
        3.0000e+01, 4.0000e+00, 1.1000e+01, 3.0000e+01, 6.0000e+00,
        1.3000e+01, 7.0000e+00, 5.0000e+00, 4.1000e+01, 3.4400e+02,
        1.0000e+00, 9.9980e+03, 1.0000e+00, 1.3139e+04, 5.9000e+01,
        1.6000e+01, 8.0000e+00, 2.3490e+03, 4.3000e+01, 3.0000e+00,
        4.2840e+03, 1.0000e+00, 1.0292e+04]),
 array([0.000e+00, 3.000e+00, 5.000e+00, 6.000e+00, 2.100e+01, 4.700e+01,
        3.000e+00, 2.600e+01, 2.980e+02, 3.000e+00, 1.400e+01, 3.100e+01,
        4.200e+01, 1.400e+01, 6.000e+00, 1.000e+00, 1.090e+02, 4.000e+00,
        5.300e+01, 1.000e+01, 3.300e+01, 1.200e+01, 3.100e+01, 6.900e+01,
        3.100e+01, 8.000e+00, 3.400e+01, 6.200e+01, 7.000e+01, 1.200e+01,
        3.100e+01, 1.730e+02, 6.100e+01, 3.000e+00, 2.200e+01, 3.140e+02,
        5.100e+01, 2.120e+02, 7.800e+01, 1.520e+02, 3.000e+00, 1.150e+02,
        8.000e+01, 3.700e+01, 2.620e+02, 3.20

In [42]:
getattr(user200_new_interval,'{}_app_interval_list'.format(user_id_list1[1]))

[array([1.0000e+00, 3.0000e+00, 2.9200e+02, 9.2300e+02, 9.0000e+00,
        5.0000e+00, 0.0000e+00, 3.0000e+00, 8.0000e+00, 3.0000e+00,
        0.0000e+00, 4.0000e+00, 2.0000e+00, 7.0000e+00, 4.0000e+00,
        4.7510e+03, 1.2600e+02, 0.0000e+00, 5.3270e+03, 0.0000e+00,
        8.7400e+02, 6.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 1.7600e+02, 5.0000e+00, 8.0500e+02, 0.0000e+00,
        2.5000e+01, 2.7000e+01, 2.9700e+02, 2.5500e+02, 1.5600e+02,
        1.4300e+02, 1.0000e+00, 2.4170e+03, 2.4000e+01, 1.3000e+01,
        1.6700e+02, 1.1669e+04, 4.6000e+01, 0.0000e+00, 2.8400e+02,
        3.8000e+01, 3.4100e+02, 2.2500e+02, 8.3200e+02, 3.0000e+00,
        2.0000e+00, 1.0000e+00, 0.0000e+00, 2.1300e+02, 0.0000e+00,
        4.0000e+00, 3.0000e+00, 5.0000e+00, 0.0000e+00, 1.2000e+01,
        7.1000e+01, 0.0000e+00, 7.4000e+01, 4.0000e+00, 2.2000e+01,
        2.0000e+00, 4.9200e+02, 2.2000e+01, 8.0000e+00, 1.0000e+00,
        2.7000e+01, 3.0480e+03, 2.0000e+00, 3.10

In [43]:
class genre_interval(object):
    def all_user_interval (self, class_name, user_list):
        gen_table = getattr(bigquery_class, '2000user_app_genre')
        gen_list = xstr(gen_table['genre'].unique())
        gen_table.loc[:,'genre'] = np.array(xstr(gen_table['genre'].tolist()))
        setattr(self, 'genre_list', gen_list)
        all_ginterval = []
        all_ginterval_count = []
        for g in gen_list:
            gen_app_list = gen_table[gen_table['genre']==g]['app_id']
            alluser_ginterval = []
            for u in user_list:
                user_app_list = getattr(class_name, '{}_app_list'.format(u))
                common_app_list =[i for i in gen_app_list if i in user_app_list]
                if len(common_app_list)==[]:
                    ginterval=[]
                else:
                    ginterval=[]
                    for a in common_app_list:
                        ginterval += getattr(class_name, '{}_{}_interval_list'.format(u,a))
                alluser_ginterval += ginterval
            all_ginterval.append(alluser_ginterval)
            all_ginterval_count.append(len(alluser_ginterval))
        setattr(self, 'genre_interval_list', all_ginterval)
        setattr(self, 'genre_interval_count_list', all_ginterval_count)

In [44]:
%%time 
genre_interval_try = genre_interval()
genre_interval_try.all_user_interval(user200_new_interval , user_id_list1)
os.system('say "your program was finished"')

CPU times: user 1min 10s, sys: 1.21 s, total: 1min 11s
Wall time: 1min 16s


0

In [88]:
getattr(genre_interval_try, 'genre_interval_list')

[[69.0,
  6.0,
  3.0,
  9.0,
  2.0,
  1.0,
  24.0,
  3417.0,
  32.0,
  338.0,
  4.0,
  49.0,
  159.0,
  3.0,
  62.0,
  2.0,
  5.0,
  34.0,
  9.0,
  3.0,
  51.0,
  1206.0,
  7.0,
  64.0,
  1085.0,
  5.0,
  29.0,
  2.0,
  2.0,
  5.0,
  4.0,
  38.0,
  17.0,
  3.0,
  13.0,
  1101.0,
  3.0,
  18.0,
  34.0,
  17.0,
  23.0,
  7.0,
  3.0,
  3.0,
  37.0,
  137.0,
  3.0,
  5.0,
  12.0,
  3.0,
  6.0,
  290.0,
  3.0,
  0.0,
  40.0,
  5.0,
  3.0,
  5.0,
  77.0,
  2.0,
  14.0,
  3.0,
  4.0,
  3.0,
  12.0,
  3.0,
  734.0,
  164.0,
  1.0,
  18.0,
  3.0,
  1.0,
  1.0,
  172.0,
  105.0,
  166.0,
  431.0,
  304.0,
  421.0,
  3.0,
  68.0,
  209.0,
  244.0,
  15.0,
  467.0,
  4.0,
  304.0,
  664.0,
  2.0,
  22.0,
  3.0,
  1.0,
  33.0,
  11.0,
  3.0,
  109.0,
  731.0,
  15.0,
  1613.0,
  50.0,
  3.0,
  0.0,
  10.0,
  34.0,
  32.0,
  117.0,
  2.0,
  3.0,
  3.0,
  4.0,
  0.0,
  14.0,
  0.0,
  11.0,
  7.0,
  98.0,
  10.0,
  0.0,
  29.0,
  2.0,
  0.0,
  16.0,
  2.0,
  27.0,
  0.0,
  23.0,
  2.0,
  0.0,
  3.0,
 

In [93]:
xstr(gen_table['genre'].unique()).index('VIDEO_PLAYERS')

48

In [94]:
rt=iforest_remove_outliner(getattr(genre_interval_try, 'genre_interval_list')[48])

In [48]:
#删除异常值 用异常分数的0.5作为阀值
def iforest_remove_outliner(intervel_list):
    sll = intervel_list
    X = np.array(sll).reshape(len(sll),1)
    clf = IsolationForest(random_state=0).fit(X)
    predict_result=clf.predict(X)
    tl=[]
    for i in range(len(sll)):
        if predict_result[i]==1:
            tl.append(sll[i])
    return tl 

In [49]:
%%time 
genre_threshold_list = []
for g in range(len(getattr(genre_interval_try, 'genre_list'))):
    genre_interval = getattr(genre_interval_try, 'genre_interval_list')[g]
    if genre_interval == []:
        genre_threshold = 'none'
        genre_threshold_list.append(genre_threshold)
    else: 
        genre_threshold = iforest_remove_outliner(genre_interval)
        genre_threshold_list.append(max(genre_threshold))
genre_threshold_list
os.system('say "your program was finished"')

CPU times: user 4min 59s, sys: 52.2 s, total: 5min 51s
Wall time: 5min 59s


0

In [50]:
getattr(bigquery_class, '2000user_app_genre')

,genre,app_id,app_count
0,NONE,com.google.android.apps.m4b,1
1,NONE,com.ghplanet.postcard,1
2,NONE,jp.co.eyecitypocket,1
3,NONE,jp.mdpr.mdprviewer,1
4,NONE,jp.japanpost.jp_bank.FIDOapp,1
...,...,...,...
8448,WEATHER,com.droid27.transparentclockweather,1
8449,WEATHER,app.nerv,1
8450,WEATHER,jp.co.yahoo.android.emg,1
8451,WEATHER,mobi.infolife.ezweather.widget.weatherandclock...,1


In [128]:
def short_session(sessionlist):
    new_session = []
    for i in range(len(sessionlist)):   
        if sessionlist[i] != sessionlist[i-1]:
            new_session.append(sessionlist[i])
        elif i==0:
            new_session.append(sessionlist[i])
    return new_session
#生成每个用户的启动顺序序列的集合
class generate_pattern_class (object):
    def generate_pattern(self, class_name, user_id): 
        app_order = getattr(class_name,'{}_app_order_list'.format(user_id))[:-1]
        interval = getattr(class_name,'{}_total_interval_list'.format(user_id))
        app_interval = [app_order, interval]
        app_interval_table = pd.DataFrame(app_interval).T
        user_app_genre = user200_app_genre 
        genre_list = []
        app_interval_table.columns=['app_id', 'interval']
        for a in app_order:
            genre_list.append(user_app_genre[user_app_genre['app_id']== a]['genre'].tolist()[0])
        #添加app的种类列
        app_interval_table['genre']=genre_list
        threshold_list = []
        for g in app_interval_table['genre'].tolist():
            idx = xstr(gen_table['genre'].unique()).index(g)
            threshold = np.array(genre_threshold_list)[idx]
            threshold_list.append(threshold)
        #添加app种类阀值列
        app_interval_table['threshold']=threshold_list    

        #当interval大于阀值时，开始新序列
        all_pattern_list = []
        all_short_pattern_list = []
        pattern_list =[]
        for l in range(len(app_interval_table)):
            if l == len(app_interval_table)-1:
                pattern_list.append(app_interval_table.iloc[l,0])
                all_pattern_list.append(pattern_list)
                all_short_pattern_list.append(short_session(pattern_list))
            elif app_interval_table.iloc[l,1] > app_interval_table.iloc[l,3]:
                pattern_list.append(app_interval_table.iloc[l,0])
                all_pattern_list.append(pattern_list)
                all_short_pattern_list.append(short_session(pattern_list))
                pattern_list =[]
            else:
                pattern_list.append(app_interval_table.iloc[l,0])
        setattr(self, '{}_all_pattern_list'.format(user_id), all_pattern_list)
        setattr(self, '{}_all_short_pattern_list'.format(user_id), all_short_pattern_list)

In [129]:
%%time
pattern_class = generate_pattern_class()
for i in range(len(user_id_list1[:10])):
    pattern_class.generate_pattern(user200_new_interval,user_id_list1[i])
os.system('say "your program was finished"')

CPU times: user 1min 55s, sys: 553 ms, total: 1min 56s
Wall time: 1min 57s


0

In [130]:
pattern_class.generate_pattern(user200_new_interval,user_id_list1[1])

In [ ]:
pattern_class.generate_pattern(user200_new_interval,user_id_list1[i])

In [132]:
getattr(pattern_class, '{}_all_short_pattern_list'.format(user_id_list1[0]))

[['com.buzzvil.honeyscreen.jp', 'com.facebook.katana'],
 ['com.facebook.katana',
  'com.buzzvil.honeyscreen.jp',
  'com.sec.android.app.launcher',
  'com.buzzvil.honeyscreen.jp',
  'com.facebook.katana',
  'com.sec.android.app.launcher'],
 ['com.sec.android.app.launcher',
  'com.facebook.katana',
  'com.sec.android.app.launcher',
  'com.buzzvil.honeyscreen.jp'],
 ['com.buzzvil.honeyscreen.jp',
  'com.sec.android.app.launcher',
  'com.buzzvil.honeyscreen.jp'],
 ['com.buzzvil.honeyscreen.jp'],
 ['com.buzzvil.honeyscreen.jp'],
 ['com.buzzvil.honeyscreen.jp'],
 ['com.buzzvil.honeyscreen.jp'],
 ['com.buzzvil.honeyscreen.jp'],
 ['com.buzzvil.honeyscreen.jp'],
 ['com.buzzvil.honeyscreen.jp'],
 ['com.buzzvil.honeyscreen.jp'],
 ['com.buzzvil.honeyscreen.jp'],
 ['com.buzzvil.honeyscreen.jp'],
 ['com.buzzvil.honeyscreen.jp'],
 ['com.buzzvil.honeyscreen.jp',
  'jp.gocro.smartnews.android',
  'com.buzzvil.honeyscreen.jp'],
 ['com.buzzvil.honeyscreen.jp',
  'jp.gocro.smartnews.android',
  'com.sec.a

In [66]:
%%time
session_list = {'user_id':[],'session':[]}
for user_id in user_id_list1:
    session_list['user_id'].append(user_id)
    session_list['session'].append(generate_pattern(user200_new_interval, user_id))
os.system('say "your program was finished"')

CPU times: user 4 µs, sys: 4 µs, total: 8 µs
Wall time: 16 µs


KeyboardInterrupt: 

In [61]:
session_list['user_id'].append(user_id_list1[1])
session_list['session'].append(generate_pattern(user200_new_interval, user_id_list1[2]))

In [ ]:
output_file = open('months.txt', 'w')

for month in months:
    output_file.write(month)

output_file.close()

In [661]:
len_list = []
for p in range(len(all_pattern_list)):
    le = len(all_pattern_list[p])
    len_list.append(le)
np.sum(len_list)

4437

In [1]:
 yoox = 880080

In [2]:
%store yoox

Stored 'yoox' (int)


In [3]:
yoox

880080

In [2]:
%store -r yoox

In [5]:
import os

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest
from tqdm import tqdm
from joblib import Parallel, delayed
FRAC = 0.25

In [8]:
user = pd.read_csv('/Users/yuxuanyang/Downloads/DSIN-master/raw_data/user_profile.csv')
sample = pd.read_csv('/Users/yuxuanyang/Downloads/DSIN-master/raw_data/raw_sample.csv')

In [9]:
if not os.path.exists('/Users/yuxuanyang/Downloads/DSIN-master/sampled_data/'):
    os.mkdir('/Users/yuxuanyang/Downloads/DSIN-master/sampled_data/')

if os.path.exists('/Users/yuxuanyang/Downloads/DSIN-master/sampled_data/user_profile_' + str(FRAC) + '_.pkl') and os.path.exists(
        '/Users/yuxuanyang/Downloads/DSIN-master/sampled_data/raw_sample_' + str(FRAC) + '_.pkl'):
    user_sub = pd.read_pickle(
        '/Users/yuxuanyang/Downloads/DSIN-master/sampled_data/user_profile_' + str(FRAC) + '_.pkl')
    sample_sub = pd.read_pickle(
        '/Users/yuxuanyang/Downloads/DSIN-master/sampled_data/raw_sample_' + str(FRAC) + '_.pkl')
else:

    if FRAC < 1.0:
        user_sub = user.sample(frac=FRAC, random_state=1024)
    else:
        user_sub = user
    sample_sub = sample.loc[sample.user.isin(user_sub.userid.unique())]
    # pd.to_pickle(user_sub, '/Users/yuxuanyang/Downloads/DSIN-master/sampled_data/user_profile_' +
    #                 str(FRAC) + '.pkl')
    # pd.to_pickle(sample_sub, '/Users/yuxuanyang/Downloads/DSIN-master/sampled_data/raw_sample_' +
    #                 str(FRAC) + '.pkl')

In [10]:
if os.path.exists('/Users/yuxuanyang/Downloads/DSIN-master/raw_data/behavior_log_pv.pkl'):
        log = pd.read_pickle('/Users/yuxuanyang/Downloads/DSIN-master/raw_data/behavior_log_pv.pkl')
# else:
#     log = pd.read_csv('/Users/yuxuanyang/Downloads/DSIN-master/raw_data/behavior_log.csv')
#     log = log.loc[log['btag'] == 'pv']
#     pd.to_pickle(log, '/Users/yuxuanyang/Downloads/DSIN-master/raw_data/behavior_log_pv.pkl')


In [ ]:
userset = user_sub.userid.unique()
log = log.loc[log.user.isin(userset)]
# pd.to_pickle(log, '/Users/yuxuanyang/Downloads/DSIN-master/sampled_data/behavior_log_pv_user_filter_' + str(FRAC) + '_.pkl')

ad = pd.read_csv('/Users/yuxuanyang/Downloads/DSIN-master/raw_data/ad_feature.csv')
ad['brand'] = ad['brand'].fillna(-1)

lbe = LabelEncoder()
# unique_cate_id = ad['cate_id'].unique()
# log = log.loc[log.cate.isin(unique_cate_id)]

unique_cate_id = np.concatenate(
    (ad['cate_id'].unique(), log['cate'].unique()))

lbe.fit(unique_cate_id)
ad['cate_id'] = lbe.transform(ad['cate_id']) + 1
log['cate'] = lbe.transform(log['cate']) + 1

lbe = LabelEncoder()
# unique_brand = np.ad['brand'].unique()
# log = log.loc[log.brand.isin(unique_brand)]

unique_brand = np.concatenate(
    (ad['brand'].unique(), log['brand'].unique()))

lbe.fit(unique_brand)
ad['brand'] = lbe.transform(ad['brand']) + 1
log['brand'] = lbe.transform(log['brand']) + 1

In [1]:
import sqlite3
import numpy as np
import pandas as pd

In [2]:
conn = sqlite3.connect('logs.db')

In [2]:
sqlite3.sqlite_version

'3.40.0'

In [15]:
a = 857237

In [16]:
pd.read_sql('''
SELECT * FROM log WHERE user = {}
'''.format(a), conn)

,user,time_stamp,btag,cate,brand
0,857237,1493816945,pv,972,84980
1,857237,1493801200,pv,5665,113748
2,857237,1493973564,pv,1141,61109
3,857237,1494595396,pv,4163,295623
4,857237,1494595384,pv,4163,295623
...,...,...,...,...,...
222,857237,1493872536,pv,5665,200028
223,857237,1493851809,pv,5665,200028
224,857237,1493872731,pv,5665,200028
225,857237,1493872205,pv,5665,200028


In [17]:
log = pd.read_sql('''
SELECT * FROM log
''', conn)

In [ ]:
log[log.user == a]